In [ ]:
# Setup
# If running in Colab, install the package first:
# !git clone https://github.com/victorlavrenko/rofa
# %cd rofa
# %pip install -e .

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from rofa import analysis
from rofa.io import download, unpack_zip


In [ ]:
# Get run artifacts
from pathlib import Path
from urllib.parse import urlparse
import os

# Mode A: local folder
RUN_DIR = ""  # e.g., '/content/drive/MyDrive/rofa_runs/greedy_20240101_120000'

# Mode B: direct release asset URL
ASSET_URL = ""  # paste a GitHub Release asset URL to download

# Optional: comparison run (greedy vs branches)
COMPARE_RUN_DIR = ""
COMPARE_ASSET_URL = ""

def _download_and_unpack(asset_url: str) -> str:
    runs_root = Path('runs')
    runs_root.mkdir(exist_ok=True)
    filename = Path(urlparse(asset_url).path).name or 'run.zip'
    zip_path = runs_root / filename
    download(asset_url, str(zip_path))
    run_dir = runs_root / zip_path.stem
    unpack_zip(str(zip_path), str(run_dir))
    return str(run_dir)

def _normalize_run_dir(run_dir: str) -> str:
    if not run_dir:
        return run_dir
    summary_path = Path(run_dir) / 'summary.jsonl'
    if summary_path.exists():
        return run_dir
    candidates = [p for p in Path(run_dir).iterdir() if p.is_dir()]
    if len(candidates) == 1 and (candidates[0] / 'summary.jsonl').exists():
        return str(candidates[0])
    return run_dir

if not RUN_DIR and ASSET_URL:
    RUN_DIR = _download_and_unpack(ASSET_URL)
RUN_DIR = _normalize_run_dir(RUN_DIR)

if not COMPARE_RUN_DIR and COMPARE_ASSET_URL:
    COMPARE_RUN_DIR = _download_and_unpack(COMPARE_ASSET_URL)
COMPARE_RUN_DIR = _normalize_run_dir(COMPARE_RUN_DIR)

print('Primary run:', RUN_DIR)
if COMPARE_RUN_DIR:
    print('Compare run:', COMPARE_RUN_DIR)


In [ ]:
# Load + parse
if not RUN_DIR:
    raise ValueError('Set RUN_DIR or ASSET_URL first.')

df = analysis.load_summary(RUN_DIR)
required_cols = {'gold'}
missing = required_cols - set(df.columns)
if missing:
    raise ValueError(f'Missing required columns: {missing}')

df_compare = None
if COMPARE_RUN_DIR:
    df_compare = analysis.load_summary(COMPARE_RUN_DIR)

df.head()


In [ ]:
# R1: greedy accuracy
greedy_accuracy = None
if 'prediction' in df.columns:
    greedy_accuracy = analysis.accuracy_greedy(df)
    print('Greedy accuracy:', greedy_accuracy)
else:
    print('Not a greedy run; skipping.')


In [ ]:
# R2: leader accuracy
leader_accuracy = None
if 'leader_correct' in df.columns:
    leader_accuracy = analysis.accuracy_leader(df)
    print('Leader accuracy:', leader_accuracy)
else:
    print('Not a branch run; skipping.')


In [ ]:
# R3: distribution of max_frac
max_frac_hist = None
if 'max_frac' in df.columns:
    max_frac_hist = analysis.max_frac_distribution(df)
    display(max_frac_hist)
else:
    print('No max_frac column; skipping.')


In [ ]:
# R4: unanimous stats
unanimous = None
if 'max_frac' in df.columns:
    unanimous = analysis.unanimous_stats(df)
    print(unanimous)
else:
    print('No max_frac column; skipping.')


In [ ]:
# R5: near-unanimous stats
near_unanimous = None
if 'max_frac' in df.columns:
    near_unanimous = analysis.near_unanimous_stats(df, threshold=0.9)
    print(near_unanimous)
else:
    print('No max_frac column; skipping.')


In [ ]:
# R6: top-2 coverage
top2_rate = None
if 'branch_preds' in df.columns:
    top2_rate = analysis.top2_coverage(df)
    print('Top-2 coverage rate:', top2_rate)
else:
    print('No branch predictions; skipping.')


In [ ]:
# R7: R/W/Other breakdown by max_frac bins
rw_other = None
if 'max_frac' in df.columns:
    rw_other = analysis.rw_other_breakdown(df)
    display(rw_other)
else:
    print('No max_frac column; skipping.')


In [ ]:
# R8: error modes (unanimous wrong)
unanimous_wrong = None
if 'max_frac' in df.columns:
    unanimous_wrong = analysis.unanimous_wrong(df)
    print('Unanimous wrong count:', len(unanimous_wrong))
    display(unanimous_wrong.head())
else:
    print('No max_frac column; skipping.')


In [ ]:
# R9: majority vote does not help (greedy vs leader)
majority_vote_gap = None
def _identify(df):
    if df is None:
        return None
    if 'prediction' in df.columns:
        return 'greedy'
    if 'leader_correct' in df.columns:
        return 'branches'
    return None

role_primary = _identify(df)
role_compare = _identify(df_compare)

if role_primary == 'greedy' and role_compare == 'branches':
    greedy_accuracy = analysis.accuracy_greedy(df)
    leader_accuracy = analysis.accuracy_leader(df_compare)
    majority_vote_gap = leader_accuracy - greedy_accuracy
elif role_primary == 'branches' and role_compare == 'greedy':
    greedy_accuracy = analysis.accuracy_greedy(df_compare)
    leader_accuracy = analysis.accuracy_leader(df)
    majority_vote_gap = leader_accuracy - greedy_accuracy

if majority_vote_gap is None:
    print('Provide both a greedy and a branches run to compare.')
else:
    print('Leader accuracy - greedy accuracy:', majority_vote_gap)


In [ ]:
# R10: subject-wise breakdown (optional)
subject_breakdown = None
if 'subject_name' in df.columns:
    accuracy_field = 'leader_correct' if 'leader_correct' in df.columns else 'is_correct'
    subject_breakdown = analysis.subject_accuracy(df, accuracy_field=accuracy_field)
    display(subject_breakdown.head(10))
else:
    print('No subject_name column; skipping.')


In [ ]:
# R11: export paper tables
from pathlib import Path
import json

run_id = Path(RUN_DIR).name if RUN_DIR else 'run'
report_dir = Path('reports') / run_id
report_dir.mkdir(parents=True, exist_ok=True)

paper_report = {
    'greedy_accuracy': greedy_accuracy,
    'leader_accuracy': leader_accuracy,
    'unanimous': unanimous,
    'near_unanimous': near_unanimous,
    'top2_coverage': top2_rate,
    'majority_vote_gap': majority_vote_gap,
}

with open(report_dir / 'paper_report.json', 'w', encoding='utf-8') as f:
    json.dump(paper_report, f, indent=2)

if max_frac_hist is not None:
    max_frac_hist.to_csv(report_dir / 'max_frac_distribution.csv')
if rw_other is not None:
    rw_other.to_csv(report_dir / 'rw_other_breakdown.csv')
if subject_breakdown is not None:
    subject_breakdown.to_csv(report_dir / 'subject_accuracy.csv')

print('Saved reports to', report_dir)


## Add your own analysis below
